In [4]:
import pandas as pd
import numpy as np

In [6]:
import pandas as pd
from datetime import datetime, timedelta

# read the data from the excel file
data = pd.read_excel('transition_data.xlsx')

# convert the Date column to a datetime object
data['Date'] = pd.to_datetime(data['Date'])

# group the data by obligor and year
grouped = data.groupby([data['Id'], data['Date'].dt.year])

# create a dictionary to store the transition counts
transitions = {}

# loop over the groups
for name, group in grouped:

    # get the obligor id and the years in the group
    obligor_id, years = name

    # sort the group by date
    group = group.sort_values(by='Date')

    # create a list to store the ratings for each year
    ratings = []

    # loop over the ratings in the group
    for i, rating in group.iterrows():

        # if this is the first rating or if the year has changed,
        # add the rating to the list of ratings
        if i == 0 or rating['Date'].year != group.iloc[i-1]['Date'].year:
            ratings.append(rating['Rating'])

    # loop over the ratings for each year and count the transitions
    for i in range(len(ratings)-1):
        start_rating = ratings[i]
        end_rating = ratings[i+1]

        # update the transition count for this transition
        if (start_rating, end_rating) in transitions:
            transitions[(start_rating, end_rating)] += 1
        else:
            transitions[(start_rating, end_rating)] = 1

# create a dataframe to store the transition matrix
matrix = pd.DataFrame(index=range(1, 9), columns=range(1, 9))

# loop over the rows and columns of the matrix
for i in range(1, 9):
    for j in range(1, 9):

        # compute the total number of obligors in the starting rating
        ni = sum([transitions[(i, k)] for k in range(1, 9)])

        # if ni is zero, set the transition probability to zero
        if ni == 0:
            matrix.at[i, j] = 0

        # otherwise, compute the transition probability
        else:
            nij = transitions.get((i, j), 0)
            pij = nij / ni
            matrix.at[i, j] = pij

# print the transition matrix
print(matrix)


IndexError: single positional indexer is out-of-bounds